In [1]:
import os
os.chdir("../")

In [2]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [3]:
resultDir = 'results'
problem = 'setcover' # choices=['setcover', 'cauctions', 'facilities', 'indset']
problemSizes = ['small', 'medium', 'big']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [104]:
# f1 = "cauctions_20210710-004536.csv"
# f2 = "cauctions_20210710-095340.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
## df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

df = pd.read_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

In [5]:
# f1 = "setcover_20210725-205924.csv"
# f2 = "setcover_20210726-163252.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
# df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

In [105]:
# eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
# eval_file = eval_files[-1]
# eval_file = f"{resultDir}/cauctions_20210710-004536.csv"

# df = pd.read_csv(eval_file)
# df_list = []
# [df[df['type'] == probSize] for probSize in problemSizes]

# df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = pd.concat([df[df['type'] == probSize] for probSize in problemSizes])
df = df.astype({'nlps': float, 'nnodes' : float})

df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()

In [106]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [107]:
dfgcnns_gmean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].mean()
dfgcnns_gmean

nnodes      stime
type   sampling_strategy                        
medium depthK             4745.136567  79.191600
       depthK2            4860.822315  80.440940
       depthK_adaptive    4772.263191  79.813774
       depthK_adaptive2   4672.397573  78.489493
       uniform5           4418.284715  74.467841

In [108]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'uniform5')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [109]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   sampling_strategy                    
medium depthK             1.073977  1.063434
       depthK2            1.100160  1.080210
       depthK_adaptive    1.080117  1.071788
       depthK_adaptive2   1.057514  1.054005
       uniform5           1.000000  1.000000

# 2. Std variances (per instance)

In [110]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','sampling_strategy','instance']).std() / df_gcnns.groupby(['type','sampling_strategy','instance']).mean()

In [111]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','sampling_strategy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   sampling_strategy                    
medium depthK             0.100748  0.087365
       depthK2            0.078302  0.073612
       depthK_adaptive    0.075314  0.062717
       depthK_adaptive2   0.151509  0.132745
       uniform5           0.090829  0.071019

# 做一下 T-Test

# 计算Wins

In [112]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'uniform5'),metric]
        for sampling_strategy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,sampling_strategy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,sampling_strategy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,sampling_strategy),(metric,)] = [t_stats, p_value]

In [113]:
ttest_res

nnodes                stime        
                            t_stats   p_value    t_stats p_value
type   sampling_strategy                                        
medium depthK             11.013343       0.0  12.230844     0.0
       depthK2              9.28446       0.0   9.995676     0.0
       depthK_adaptive     8.522014       0.0  10.707979     0.0
       depthK_adaptive2     4.78778  0.000003   6.028568     0.0
       uniform5                 0.0       1.0        0.0     1.0

In [114]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type    instance                                                          seed
medium  data/instances/setcover/transfer_1000r_1000c_0.05d/instance_1.lp  0         5.0
                                                                          1         6.0
                                                                          2        12.0
                                                                          3         3.0
                                                                          4         9.0
        data/instances/setcover/transfer_1000r_1000c_0.05d/instance_2.lp  0        35.0
                                                                          1        36.0
                                                                          2        27.0
                                                                          3        48.0
                                                                          4        39.0
        data/instances/setcover/transfer_

In [115]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [116]:
len(winner_idx)

35

In [117]:
dfgcnns_wins = df_gcnns.groupby(['type', 'sampling_strategy'])['wins'].sum()
dfgcnns_wins

type    sampling_strategy
medium  depthK               10
        depthK2               4
        depthK_adaptive       2
        depthK_adaptive2      8
        uniform5             11
Name: wins, dtype: int64

In [118]:
stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

Ttest_indResult(statistic=-2.455047672770633, pvalue=0.014949535302005438)

# 把Means、std 和 Wins合并起来

In [119]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std           wins
                            nnodes     stime    nnodes     stime wins
type   sampling_strategy                                             
medium depthK             1.073977  1.063434  0.100748  0.087365   10
       depthK2            1.100160  1.080210  0.078302  0.073612    4
       depthK_adaptive    1.080117  1.071788  0.075314  0.062717    2
       depthK_adaptive2   1.057514  1.054005  0.151509  0.132745    8
       uniform5           1.000000  1.000000  0.090829  0.071019   11

In [120]:
# output_idx = pd.MultiIndex.from_product((dfgcnns_gmean_norm.index.get_level_values('type').unique(), sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx = pd.MultiIndex.from_product((problemSizes, sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx

MultiIndex([('medium',         'uniform5'),
            ('medium',           'depthK'),
            ('medium',          'depthK2'),
            ('medium',  'depthK_adaptive'),
            ('medium', 'depthK_adaptive2')],
           names=['type', 'sampling_strategy'])

In [121]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [122]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   sampling_strategy                                  
medium uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN
       depthK_adaptive      NaN   NaN  NaN             NaN
       depthK_adaptive2     NaN   NaN  NaN             NaN

In [123]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [124]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [125]:
out_df1

nnodes             stime  wins  \
type   sampling_strategy                                             
medium uniform5            1.0000r ± 9.08%   1.0000r ± 7.10%    11   
       depthK             1.0740r ± 10.07%   1.0634r ± 8.74%    10   
       depthK2             1.1002r ± 7.83%   1.0802r ± 7.36%     4   
       depthK_adaptive     1.0801r ± 7.53%   1.0718r ± 6.27%     2   
       depthK_adaptive2   1.0575r ± 15.15%  1.0540r ± 13.27%     8   

                         t-stats (p-val)  
type   sampling_strategy                  
medium uniform5             0.00(1.0000)  
       depthK              12.23(0.0000)  
       depthK2             10.00(0.0000)  
       depthK_adaptive     10.71(0.0000)  
       depthK_adaptive2     6.03(0.0000)